# Save and load the elicit object

## Save an unfitted `elicit` object (e.g., for sharing with collaborators)

### Step 0: Load necessary libraries and functions/classes

In [11]:
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import elicit as el

from elicit.extras import utils

tfd = tfp.distributions

# numeric, standardized predictor
def std_predictor(N, quantiles):
    X = tf.cast(np.arange(N), tf.float32)
    X_std = (X-tf.reduce_mean(X))/tf.math.reduce_std(X)
    X_sel = tfp.stats.percentile(X_std, quantiles)
    return X_sel

# implemented, generative model
class ToyModel:
    def __call__(self, prior_samples, design_matrix, **kwargs):
        B = prior_samples.shape[0]
        S = prior_samples.shape[1]

        # preprocess shape of design matrix
        X = tf.broadcast_to(design_matrix[None, None,:],
                           (B,S,len(design_matrix)))
        # linear predictor (= mu)
        epred = tf.add(prior_samples[:, :, 0][:,:,None],
                       tf.multiply(prior_samples[:, :, 1][:,:,None], X)
                       )
        # data-generating model
        likelihood = tfd.Normal(
            loc=epred, scale=tf.expand_dims(prior_samples[:, :, -1], -1)
        )
        # prior predictive distribution (=height)
        ypred = likelihood.sample()
        
        # selected observations
        y_X0, y_X1, y_X2 = (ypred[:,:,0], ypred[:,:,1], ypred[:,:,2])

        return dict(
            likelihood=likelihood,
            ypred=ypred, epred=epred,
            prior_samples=prior_samples,
            y_X0=y_X0, y_X1=y_X1, y_X2=y_X2
        )

### Step 1: Create `elicit` object

In [2]:
elicit = el.Elicit(
    model=el.model(
        obj=ToyModel,
        design_matrix=std_predictor(N=200, quantiles=[25,50,75])
        ),
    parameters=[
        el.parameter(
            name="beta0",
            family=tfd.Normal,
            hyperparams=dict(
                loc=el.hyper("mu0"),
                scale=el.hyper("sigma0", lower=0)
                )
        ),
        el.parameter(
            name="beta1",
            family=tfd.Normal,
            hyperparams=dict(
                loc=el.hyper("mu1"),
                scale=el.hyper("sigma1", lower=0) # TODO specify error message
                )
        ),
        el.parameter(
            name="sigma",
            family=tfd.HalfNormal,
            hyperparams=dict(
                scale=el.hyper("sigma2", lower=0)
                )
        ),
    ],
    targets=[
        el.target(
            name="y_X0",
            query=el.queries.quantiles((5, 25, 50, 75, 95)),
            loss=el.losses.MMD2(kernel="energy"),
            weight=1.0
        ),
        el.target(
            name="y_X1",
            query=el.queries.quantiles((5, 25, 50, 75, 95)),
            loss=el.losses.MMD2(kernel="energy"),
            weight=1.0
        ),
        el.target(
            name="y_X2",
            query=el.queries.quantiles((5, 25, 50, 75, 95)),
            loss=el.losses.MMD2(kernel="energy"),
            weight=1.0
        )
    ],
    expert=el.expert.simulator(
        ground_truth = {
            "beta0": tfd.Normal(loc=5, scale=1),
            "beta1": tfd.Normal(loc=2, scale=1),
            "sigma": tfd.HalfNormal(scale=10.0),
        },
        num_samples = 10_000
    ),
    optimizer=el.optimizer(
        optimizer=tf.keras.optimizers.Adam,
        learning_rate=0.1,
        clipnorm=1.0
        ),
    trainer=el.trainer(
        method="parametric_prior",
        name="toy0",
        seed=0,
        epochs=4
    ),
    initializer=el.initializer(
        method="sobol",
        loss_quantile=0,
        iterations=8,
        distribution=el.initialization.uniform(
            radius=1,
            mean=0
            )
        )
)

### Step 2: Save unfitted `elicit` object

In [3]:
el.utils.save_elicit(elicit, "./results/elicit_toy0.pkl")

saved elicit as: ./results/elicit_toy0.pkl


## Load and fit *unfitted* `elicit` object

### Load `elicit` object

In [4]:
elicit_toy0 = el.utils.load_elicit("./results/elicit_toy0.pkl")

### Fit loaded `elicit` object

In [5]:
elicit_toy0.fit(save_dir=None, silent=True)

Initialization


100%|██████████| 8/8 [00:01<00:00,  5.38it/s]


 
Training


100%|██████████| 4/4 [00:01<00:00,  2.38it/s]


### Inspect fitted `elicit` object

In [6]:
elicit_toy0.history["loss"]

[<tf.Tensor: shape=(), dtype=float32, numpy=10.571016>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.069366>,
 <tf.Tensor: shape=(), dtype=float32, numpy=9.578753>,
 <tf.Tensor: shape=(), dtype=float32, numpy=9.098384>]

## Save and reload fitted `elicit` object
### Step 1: Save fitted object

In [7]:
el.utils.save_elicit(elicit_toy0, "./results/fit_toy0.pkl")

saved elicit as: ./results/fit_toy0.pkl


### Step 2: Load fitted object and inspect it

In [8]:
fit_toy0 = el.utils.load_elicit("./results/fit_toy0.pkl")
fit_toy0.history["loss"]

[<tf.Tensor: shape=(), dtype=float32, numpy=10.571016>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.069366>,
 <tf.Tensor: shape=(), dtype=float32, numpy=9.578753>,
 <tf.Tensor: shape=(), dtype=float32, numpy=9.098384>]

## Q&A

### What happens when I want to fit an already fitted `elicit` object?

In [9]:
fit_toy0.fit(save_dir=None)

# prompt: 
# elicit object is already fitted. 
# Do you want to fit it again and overwrite the results? 
# Press 'n' to stop process and 'y' to continue fitting. 

# user input: n

elicit object is already fitted. Do you want to fit it again and overwrite the results? Press 'n' to stop process and 'y' to continue fitting. n


'Process aborded; elicit object is not re-fitted.'

#### Can I force re-fitting?

In [ ]:
fit_toy0.fit(save_dir=None, force_fit=True)

### What happens when I want to save an elicit object with a name that already exists?

In [10]:
el.utils.save_elicit(elicit_toy0, "./results/fit_toy0.pkl")

# prompt:
# In provided directory exists already a file with identical name. Do you want to overwrite it? 
# Press 'y' for overwriting and 'n' for abording. 

# user input: n

In provided directory exists already a file with identical name. Do you want to overwrite it? Press 'y' for overwriting and 'n' for abording. n


'Process aborded. File is not overwritten.'

#### Can I force overwriting of file while saving?

In [ ]:
el.utils.save_elicit(elicit_toy0, "./results/fit_toy0.pkl", force_overwrite=True)